In [ ]:
%cd '/content/HW4'

/content/drive/MyDrive/Colab Notebooks/HW4


In [1]:
# imports

from pathlib import Path
from omegaconf import OmegaConf

from src.data_pipeline import CIFAR10Pipeline
from src.runner import ExperimentRunner

In [2]:
# load config

cfg = OmegaConf.load("conf/config.yaml")

print("="*30)
print("Setting up data pipeline...")
print("="*30)

data_pipeline = CIFAR10Pipeline(cfg)
train_loader, val_loader, test_loader = data_pipeline.setup()

print(f"\nDataset: CIFAR-10")
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")
print(f"Batch size: {cfg.train.batch_size}\n")

Setting up data pipeline...

Dataset: CIFAR-10
Training samples: 45000
Validation samples: 5000
Test samples: 10000
Batch size: 128



In [3]:
# Auxiliary functions

def print_exp_params(cfg, exp_name):

  model_name = cfg.model.model_name

  print("="*30)
  print(f"Experiment: {exp_name}")
  print("="*30)
  print(f"  Mode: {cfg.mode}")
  print(f"  Model: {cfg.model.model_name}")
  print(f"  Optimizer: {cfg.train.optimizer}")
  print(f"  Learning Rate: {cfg.train.learning_rate}")
  print(f"  Batch Size: {cfg.train.batch_size}")
  print(f"  Epochs: {cfg.train.num_epochs}")
  print(f"  Weight Decay: {cfg.train.weight_decay}")

  if model_name.lower() == 'customcnn':
      print(f"  CNN Layers: {cfg.model.cnn.num_layers}")
      print(f"  Use BN: {cfg.model.cnn.use_batch_norm}")
      print(f"  Use Dropout: {cfg.model.cnn.use_dropout}")

      if cfg.model.cnn.use_dropout:
        print(f"  Dropout Rate: {cfg.model.cnn.dropout_rate}")

      print(f"  FC Layers: {cfg.model.cnn.fc_layers}")


def build_exp_name(exp_config):

  model_name = exp_config['model']['model_name']
  
  if model_name.lower() == 'customcnn':
    cnn = exp_config['model']['cnn']
    train = exp_config['train']
    
    num_conv_layers = len(cnn['num_layers'])
    arch_type = 'Shallow' if num_conv_layers <= 2 else 'Deep'
    
    conv_str = cnn['num_layers']
    fc_str = cnn['fc_layers']
    lr = train['learning_rate']
    
    # Build name
    name = f"{arch_type}_convs={conv_str}_fc={fc_str}_bz={train['batch_size']}_opt={train['optimizer']}_lr={lr}"

    return name
  
  # For ResNet models
  else:
    return model_name

In [4]:
# initialize experiment params

cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

# Dictionary to store results from all experiments
experiment_results = {}

# Run experiment for the configured model
exp_name = build_exp_name(cfg)
print_exp_params(cfg, exp_name)


Experiment: ResNet18
  Mode: pretrained
  Model: ResNet18
  Optimizer: adam
  Learning Rate: 0.001
  Batch Size: 128
  Epochs: 1
  Weight Decay: 0.0001


In [ ]:
# run single experiment

# define experiment name as you wish
exp_name = cfg.model.model_name

runner = ExperimentRunner(
    cfg=cfg,
    exp_name=exp_name,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    class_names=cifar10_classes
)

results = runner.run()

experiment_results[results['model_name']] = results

print("\n" + "="*30)
print("Experiment completed successfully!")
print("="*30)

In [ ]:
# Load and evaluate existing model weights

# Example: Load weights using config
# Set cfg.load_weights = True and cfg.mode to match the model you want to load
cfg.load_weights = True
cfg.mode = 'pretrained'  # or 'scratch'
cfg.train.freeze = False  # If loading pretrained model, set freeze accordingly

# Create runner with load_weights=True
runner = ExperimentRunner(
    cfg=cfg,
    exp_name=cfg.model.model_name,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    class_names=cifar10_classes,
    load_weights=True
)

# Check if weights file exists
print(f"Model path: {runner.weights_path}")
print(f"Model exists: {Path(runner.weights_path).exists()}")

# Run evaluation (will skip training and load weights)
if Path(runner.weights_path).exists():
    results = runner.run()
    experiment_results[results['model_name']] = results

    print("\n" + "="*60)
    print("Model evaluation completed!")
    print("="*60)
else:
    print(f"\nNo saved model found at {runner.weights_path}")
    print("Train a model first or check your config parameters.")

## Experiments

We perform the following experiments for each architecture (Shallow vs Deep)
1. Architecture Params
1. Batch Size (64 vs 128)
2. Optimizer & lr (adam vs SGD, 1e-3 vs 1e-4)
3. Regularization (Dropout vs BN vs Both)

### Exp1 - Architecture

In [ ]:
# define 1st experiment params
experiments = [
    
    # Shallow CNN from scratch

    # Single layer - very shallow
    {
        'mode': 'scratch',
        'load_weights': False,
        'model': {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [64],
            'kernel_size': [3],
            'stride': [1],
            'fc_layers': [256],
            'use_dropout': False,
            'dropout_rate': 0.4,
            'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': 'false'
        }
    },

    # Two layers - standard shallow
    {
        'mode': 'scratch',
        'load_weights': False,
        'model': {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [64, 128],
            'kernel_size': [3, 3],
            'stride': [1, 1],
            'fc_layers': [256, 128],
            'use_dropout': False,
            'dropout_rate': 0.4,
            'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': 'false'
        }
    },

    # Two layers - wider shallow (more channels)
    {
        'mode': 'scratch',
        'load_weights': False,
        'model': {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [128, 256],
            'kernel_size': [3, 3],
            'stride': [1, 1],
          'fc_layers': [512],
          'use_dropout': False,
          'dropout_rate': 0.4,
          'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': False
        }
    },

    # Deep CNN from scratch

    # Four layers - standard deep
    {
        'mode': 'scratch',
        'load_weights': False,
        'model': {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [32, 64, 128, 256],
            'kernel_size': [3, 3, 3, 3],
            'stride': [1, 1, 1, 1],
            'fc_layers': [512, 128],
            'use_dropout': False,
            'dropout_rate': 0.4,
            'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': False
        }
    },    

    # Five layers - deeper
    {
        'mode': 'scratch',
        'load_weights': False,
        'model':
        {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [32, 64, 64, 128, 256],
            'kernel_size': [3, 3, 3, 3, 3],
            'stride': [1, 1, 1, 1, 1],
            'fc_layers': [512, 256, 128],
            'use_dropout': False,
            'dropout_rate': 0.4,
            'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': False
        }
    },

    # Five layers - VGG-style (double channels progressively)
    {
        'mode': 'scratch',
        'load_weights': False,
        'model': {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [64, 128, 256, 512, 512],
            'kernel_size': [3, 3, 3, 3, 3],
            'stride': [1, 1, 1, 1, 1],
            'fc_layers': [1024, 512],
            'use_dropout': False,
            'dropout_rate': 0.4,
            'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': False
        }
    },
]

In [ ]:
# run multiple experiments

cfg.results_dir = 'results/Exp1_Architecture'
cfg.checkpoints_dir = 'checkpoints/Exp1_Architecture'
for exp in experiments:
  
  exp_name = build_exp_name(exp)

  cfg.mode = exp['mode']
  cfg.model.model_name = exp['model']['model_name']

  if 'train' in exp:
    for key in exp['train'].keys():
      cfg['train'][key] = exp['train'][key]
      
  if 'cnn' in exp:
    for key in exp['cnn'].keys():
      cfg['model']['cnn'][key] = exp['cnn'][key]

  # reload data for new config
  data_pipeline = CIFAR10Pipeline(cfg)
  train_loader, val_loader, test_loader = data_pipeline.setup()
  
  print_exp_params(cfg, exp_name)

  runner = ExperimentRunner(
      cfg=cfg,
      exp_name=exp_name,
      train_loader=train_loader,
      val_loader=val_loader,
      test_loader=test_loader,
      class_names=cifar10_classes,
      load_weights=exp['load_weights']
  )

  results = runner.run()
  experiment_results[exp_name] = results

# Compare all experiments
ExperimentRunner.compare(experiment_results, save_dir="results")

### Exp2 - Batch Size

We took the best models from Exp1 for each network based on F1 score.

In [5]:
# define 2nd experiment params
experiments = [
    
    # Shallow CNN from scratch
    {
        'mode': 'scratch',
        'load_weights': False,
        'model': {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [64, 128],
            'kernel_size': [3, 3],
            'stride': [1, 1],
            'fc_layers': [256, 128],
            'use_dropout': False,
            'dropout_rate': 0.4,
            'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 32,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': 'false'
        }
    },

  
    # Deep CNN from scratch
    {
        'mode': 'scratch',
        'load_weights': False,
        'model': {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [64, 128, 256, 512, 512],
            'kernel_size': [3, 3, 3, 3, 3],
            'stride': [1, 1, 1, 1, 1],
            'fc_layers': [1024, 512],
            'use_dropout': False,
            'dropout_rate': 0.4,
            'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': False
        }
    },
]

In [ ]:
# run multiple experiments

cfg.results_dir = 'results/Exp2_BatchSize'
cfg.checkpoints_dir = 'checkpoints/Exp2_BatchSize'
for exp in experiments:

  for bz in [32, 64, 128, 256]:

    exp['train']['batch_size'] = bz
    cfg.train.batch_size = bz
    exp_name = build_exp_name(exp)

    cfg.mode = exp['mode']
    cfg.model.model_name = exp['model']['model_name']

    if 'train' in exp:
      for key in exp['train'].keys():
        cfg['train'][key] = exp['train'][key]
        
    if 'cnn' in exp:
      for key in exp['cnn'].keys():
        cfg['model']['cnn'][key] = exp['cnn'][key]

    # reload data for new config
    data_pipeline = CIFAR10Pipeline(cfg)
    train_loader, val_loader, test_loader = data_pipeline.setup()
    
    print_exp_params(cfg, exp_name)

    runner = ExperimentRunner(
        cfg=cfg,
        exp_name=exp_name,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        class_names=cifar10_classes,
        load_weights=exp['load_weights']
    )

    results = runner.run()
    experiment_results[exp_name] = results

# Compare all experiments
ExperimentRunner.compare(experiment_results, save_dir="results")

### Exp3 - Optimizer

Batch sizes 32 and 64 gave the best results. We will test the optimizers with either one.

In [9]:
# define 3nd experiment params
experiments = [
    
    # Shallow CNN from scratch
    {
        'mode': 'scratch',
        'load_weights': False,
        'model': {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [64, 128],
            'kernel_size': [3, 3],
            'stride': [1, 1],
            'fc_layers': [256, 128],
            'use_dropout': False,
            'dropout_rate': 0.4,
            'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 32,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': 'false'
        }
    },

  
    # Deep CNN from scratch
    {
        'mode': 'scratch',
        'load_weights': False,
        'model': {
          'model_name': 'CustomCNN',
          'cnn': {
            'num_layers': [64, 128, 256, 512, 512],
            'kernel_size': [3, 3, 3, 3, 3],
            'stride': [1, 1, 1, 1, 1],
            'fc_layers': [1024, 512],
            'use_dropout': False,
            'dropout_rate': 0.4,
            'use_batch_norm': False
          }
        },
        'train': {
          'batch_size': 128,
          'learning_rate': 0.001,
          'num_epochs': 20,
          'optimizer': 'adam',
          'weight_decay': 0.0001,
          'momentum': 0.9,
          'early_stopping_patience': 3,
          'freeze': False
        }
    },
]

In [ ]:
# run multiple experiments

cfg.results_dir = 'results/Exp3_Optimizer'
cfg.checkpoints_dir = 'checkpoints/Exp3_Optimizer'
for exp in experiments:

  for bz in [32, 64]:
    
    for opt in ['adam', 'sgd']:
      
      for lr in [0.001, 0.0001]:

        exp['train']['batch_size'] = bz
        exp['train']['optimizer'] = opt
        exp['train']['learning_rate'] = lr

        cfg.train.batch_size = bz
        cfg.train.optimizer = opt
        cfg.train.learning_rate = lr
        exp_name = build_exp_name(exp)

        cfg.mode = exp['mode']
        cfg.model.model_name = exp['model']['model_name']

        if 'train' in exp:
          for key in exp['train'].keys():
            cfg['train'][key] = exp['train'][key]
            
        if 'cnn' in exp:
          for key in exp['cnn'].keys():
            cfg['model']['cnn'][key] = exp['cnn'][key]

        # reload data for new config
        data_pipeline = CIFAR10Pipeline(cfg)
        train_loader, val_loader, test_loader = data_pipeline.setup()
        
        print_exp_params(cfg, exp_name)

        runner = ExperimentRunner(
            cfg=cfg,
            exp_name=exp_name,
            train_loader=train_loader,
            val_loader=val_loader,
            test_loader=test_loader,
            class_names=cifar10_classes,
            load_weights=exp['load_weights']
        )

        results = runner.run()
        experiment_results[exp_name] = results

# Compare all experiments
ExperimentRunner.compare(experiment_results, save_dir="results")

Experiment: Shallow_convs=[64, 128]_fc=[256, 128]_bz=32_opt=adam_lr=0.001
  Mode: scratch
  Model: CustomCNN
  Optimizer: adam
  Learning Rate: 0.001
  Batch Size: 32
  Epochs: 20
  Weight Decay: 0.0001
  CNN Layers: [32, 64]
  Use BN: True
  Use Dropout: True
  Dropout Rate: 0.3
  FC Layers: [128]

Initializing CustomCNN...
Mode: scratch
Built CustomCNN with 29,194 parameters

Training for 20 epochs...


Epoch [1/20] Train Loss: 1.9554 Acc: 25.46% | Val Loss: 1.7206 Acc: 36.98%


Epoch [2/20] Train Loss: 1.8291 Acc: 30.61% | Val Loss: 1.6579 Acc: 36.68%


Epoch [3/20] Train Loss: 1.7861 Acc: 32.27% | Val Loss: 1.6048 Acc: 39.34%


Epoch [4/20] Train Loss: 1.7435 Acc: 34.52% | Val Loss: 1.5831 Acc: 42.08%


Epoch [5/20] Train Loss: 1.7118 Acc: 36.12% | Val Loss: 1.5449 Acc: 43.72%


Epoch [6/20] Train Loss: 1.6767 Acc: 37.56% | Val Loss: 1.5032 Acc: 45.94%


KeyboardInterrupt: 